<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1/TP1_Parte3_Java_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile Server.java
import java.io.Closeable;
import java.io.IOException;
import java.net.InetAddress;
import java.net.ServerSocket;
import java.net.Socket;
import java.io.DataInputStream;
import java.io.ObjectOutputStream;

public class Server
{
  private static final int port = 4444;
  private static final int backlog = 5;
  private static ServerSocket serverSocket = null;

  public static void main(String[] args)
  {
    runServer();
  }

  public static void runServer()
  {
    try
    {
      //Creación del socket.
      serverSocket = new ServerSocket(port, backlog, InetAddress.getLocalHost());
      System.out.println("Socket creado. IP: " + InetAddress.getLocalHost() + " - Puerto: " + port);

      //Esperamos conexiones indefinidamente.
      waitForConnection();
      
    }
    catch (IOException e)
    {
      System.out.println(e.getStackTrace());
      System.exit(1);
    }
    finally
    {
      closeResource(serverSocket);
    }
  }

  private static void waitForConnection()
  {
    try
    {
      System.out.println("Esperando conexión");
      final Socket socket = serverSocket.accept();
      System.out.println("Conexión recibida desde: " + socket.getRemoteSocketAddress());
      Thread clientHandler = new Thread(() -> handleClient(socket));
      clientHandler.start();
    }
    catch (IOException e)
    {
      System.out.println("Error en la conexión");
    }
  }

  private static void handleClient(Socket socket)
  {
    try
    {
      DataInputStream dataInputStream = new DataInputStream(socket.getInputStream());
      ObjectOutputStream objectOutputStream = new ObjectOutputStream(socket.getOutputStream());

      //Leer la info del cliente.
      String input = dataInputStream.readUTF();

      //Procesar info.
      StringInfo info = StringInfo.getStringInfo(input);

      //Enviar resultados al cliente.
      objectOutputStream.writeObject(info);
    }
    catch (IOException e)
    {
      System.out.println("Se interrumpió la conexión con el socket.");
    }
    finally
    {
      closeResource(socket);
    }
  }

  private static void closeResource(Closeable resource)
  {
    if (resource != null)
    {
      try
      {
        resource.close();
      }
      catch (IOException e)
      {
        System.out.println("Error al cerrar el recurso.");
        ;
      }
    }
  }
}

Overwriting Server.java


In [ ]:
%%writefile Client.java
import java.io.DataOutputStream;
import java.io.IOException;
import java.io.ObjectInputStream;
import java.net.InetAddress;
import java.net.Socket;

public class Client
{
  private static final int port = 4444;

  public static void main(String[] args)
  {
    Socket socket = null;

    //Verificar entrada
    if (args.length < 1)
    {
      System.out.println("Debe indicar una palabra a analizar por parametro");
      System.exit(1);
    }
    try
    {
      //Crear sockets y streams de entrada y salida.
      socket = new Socket(InetAddress.getLocalHost(), port);
    }
    catch (IOException e)
    {
      System.out.println("El servidor no se encuentra disponible");
      System.exit(1);
    }

    try
    {
      DataOutputStream dataOutputStream = new DataOutputStream(socket.getOutputStream());
      ObjectInputStream objectInputStream = new ObjectInputStream(socket.getInputStream());

      //Enviar el mensaje a procesar.
      dataOutputStream.writeUTF(args[0]);

      //Leer respuesta del servidor.
      StringInfo returnMessage = (StringInfo) objectInputStream.readObject();

      System.out.println(returnMessage);
    }
    catch (IOException e)
    {
      System.out.println("Error en conexión");
      System.out.println(e.getStackTrace());
    }
    catch (ClassNotFoundException e)
    {
      System.out.println("El mensaje recibido no tiene el formato esperado.");
    }
    finally
    {
      if (socket != null)
      {
        try
        {
          socket.close();
        }
        catch (IOException e)
        {
          throw new RuntimeException(e);
        }
      }
    }
  }
}

Overwriting Client.java


In [ ]:
%%writefile StringInfo.java
import java.io.Serializable;

public class StringInfo implements Serializable
{
  private int length;
  private int lettersCount;
  private int digitsCount;
  private int specialCharsCount;

  private StringInfo(int length)
  {
    this.length = length;
    lettersCount = 0;
    digitsCount = 0;
    specialCharsCount = 0;
  }

  private void addLetter()
  {
    lettersCount++;
  }

  private void addDigit()
  {
    digitsCount++;
  }

  private void addSpecialChar()
  {
    specialCharsCount++;
  }

  private void processChar(char c)
  {
    if (Character.isLetter(c))
    {
      addLetter();
    }
    else if (Character.isDigit(c))
    {
      addDigit();
    }
    else
    {
      addSpecialChar();
    }
  }

  public static StringInfo getStringInfo(String name)
  {
    StringInfo info = new StringInfo(name.length());
    for (char c : name.toCharArray())
    {
      info.processChar(c);
    }
    return info;
  }

  public int getLength()
  {
    return length;
  }

  public int getLettersCount()
  {
    return lettersCount;
  }

  public int getDigitsCount()
  {
    return digitsCount;
  }

  public int getSpecialCharsCount()
  {
    return specialCharsCount;
  }

  @Override
  public String toString()
  {
    return new StringBuilder()
        .append("Cantidad de caracteres totales: ").append(getLength()).append(System.lineSeparator())
        .append("Cantidad de letras: ").append(getLettersCount()).append(System.lineSeparator())
        .append("Cantidad de dígitos: ").append(getDigitsCount()).append(System.lineSeparator())
        .append("Cantidad de caracteres que no son letra ni dígito: ").append(getSpecialCharsCount())
        .append(System.lineSeparator())
        .toString();
  }
}

Overwriting StringInfo.java


Como las clases las tenemos en archivos separados, es necesario compilarlas antes de ejecutar el programa.

In [ ]:
!javac Server.java
!javac Client.java
!javac StringInfo.java

Intentamos ejecutar el cliente sin haber ejecutado el servidor

In [ ]:
!java Client "¡Hol4 Mund0!"

El servidor no se encuentra disponible


Intentamos ejecutar el cliente sin el parametro necesario


In [ ]:
!nohup java Server 1>salidaJava 2>/dev/null &
!java Client

Debe indicar una palabra a analizar por parametro


Iniciamos el server en background, redirigiendo su saalida a un archivo para poder verificarla en caso de ser necesario.
Iniciamos el cliente en primer plano para ver los resultados por pantalla.

Enviamos a procesar el mensaje "¡Hol4 Mund0!", el cual tiene:
*   12 caracteres totales
*   7 letras
*   2 dígitos
*   3 caracteres especiales






In [ ]:
!nohup java Server 1>salidaJava 2>/dev/null &
!java Client "¡Hol4 Mund0!"

Cantidad de caracteres totales: 12
Cantidad de letras: 7
Cantidad de dígitos: 2
Cantidad de caracteres que no son letra ni dígito: 3

